### logging

In [ ]:
if globals().get('LOGGING_LEVEL') == None:
    LOGGING_LEVEL=2

### module import protection

In [ ]:
if globals().get('LOADED_VIEWING_TABS') == None:
    if LOGGING_LEVEL > 0: print('LOADED_VIEWING_TABS')
    LOADED_VIEWING_TABS=True

### modules

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
# %run DATAFRAME.ipynb

### imports

##### matplotlib

In [ ]:
# imports
%matplotlib widget 

# plot
## a.plot(method='pandas.matplotlib')

##### plotly

In [ ]:
# imports
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# plot
## a.plot(method='cufflinks') # WORKS

In [ ]:
# imports

# plot
## a.plot(method='pandas.plotly')

In [ ]:
# imports
import plotly.express as px

# plot
## a.plot(method='px')

##### bokeh

In [ ]:
# imports
import pandas_bokeh
pandas_bokeh.output_notebook(hide_banner=True) # output in cell

# plot
## a.plot(method='pandas.pandas_bokeh')

In [ ]:
# imports

# plot
## a.plot(method='pandas.holoviews')

In [ ]:
'''
dat3.plot(backend='pandas_bokeh')
dat3.plot(backend='holoviews') # works outside of OUTPUT widget
'''
None

In [ ]:
from ipywidgets import Text,Output,GridspecLayout,GridBox # for SINGLE_TAB
from ipywidgets import Tab,Layout # for TAB_COLLECTION

### begin

In [ ]:
'''
doesnt contain data
only explains how to generate a plot off a pd.DataFrame
'''
class SINGLE_PLOT:
    
    _plot_methods=(
        'cufflinks',           # plotly           # WORKS
        'px',                  # plotly           # WORKS
        'bokeh',               # bokeh            # TODO
        'pandas.matplotlib',   # matplotlib       # WORKS = NEEDS `%matplotlib widget` MAGIC
        'pandas.plotly',       # plotly express   # WORKS = 
        'pandas.pandas_bokeh', # bokeh            # NOPE  = BOKEH DOESNT WORK IN OUTPUT
        'pandas.holoviews',    # bokeh            # NOPE  = BOKEH DOESNT WORK IN OUTPUT
        'pandas.altair',       # altair           # NOPE  = DOESNT WORK
    )
    
    def __init__(self,
        name                : str              = 'SINGLE_PLOT',
        description         : str              = 'SINGLE_PLOT_DESCRIPTION',
        method              : str              = 'default',
        query               : str              = 'some query',
        ):

        self._logging_level                    = LOGGING_LEVEL                  # GLOBAL VARIABLE
        
        # attributes
        self.name           : str              = name
        self.description    : str              = name
        self._method        : str              = method if method != None else 'else'
        self._query         : str              = query
            
        self._times_drawn   : int              = 0
        self.widget                            = Output()
    
    '''
# should be same = matplotlib
a.plot(method='pandas.matplotlib') # works
        
# should be same = plotly
a.plot(method='cufflinks') # works
a.plot(method='pandas.plotly') # works, but cant suppress legend
a.plot(method='px') # works
        
# should be same = bokeh
a.plot(method='pandas.pandas_bokeh') # works
a.plot(method='pandas.holoviews')
    '''
    
    # extraday query
    def _extraday_query(self,df : pd.DataFrame):
        return df[self._query].unstack()
    
    # intraday query
    def _intraday_query(self,df : pd.DataFrame):
        pass # TODO
        # return df[self._query].groupby(['sym','minute']).mean().unstack() # TODO
    
    # factor query
    def _intraday_query(self,df : pd.DataFrame,x,y): # TODO
        pass
    
    # clear and redraw
    def update_plot(self,
        df     : pd.DataFrame = pd.DataFrame(),
        method : str          = None,
        ) -> None :
        # do nothing if no query provided
        if self._query is None: 
            return
        
        # ensure method populated
        if method is None: 
            method = self._method
            
        self._times_drawn+=1 # increment counter
        self.widget.clear_output() # clear image
        
        # bail if nothing to plot
        if len(df)==0:
           return
    
        with self.widget:
            if self._logging_level > 0: print(self.name,'_plot_method',':',method)
        
            if method in SINGLE_PLOT._plot_methods:
                if method in [
                    'pandas.matplotlib',   # matplotlib # NEED `%matplotlib widget` MAGIC FOR IT TO WORK
                    'pandas.plotly',       # plotly express
                    'pandas.pandas_bokeh', # bokeh # BOKEH DOESNT WORK IN OUTPUT
                    'pandas.holoviews',    # bokeh # BOKEH DOESNT WORK IN OUTPUT
                    'pandas.altair',       # altair # DOESNT WORK
                ]:
                    plotting_backend=method.split('.')[1]
                    if self._logging_level > 1: print('backend',plotting_backend)
                        
                    if plotting_backend in [
                        'matplotlib',
                    ]:
                        out=self._extraday_query(df).plot(backend=plotting_backend,title=self._query,legend=False)
                        display(out)
                    elif plotting_backend in [
                        'plotly',
                    ]:
                        out=self._extraday_query(df).plot(backend=plotting_backend,title=self._query) # dont know how to suppress legend
                        display(out)
                    elif plotting_backend in [
                        'pandas_bokeh', # bokeh
                    ]:
                        out=self._extraday_query(df).plot(backend=plotting_backend,title=self._query,legend=False)
                        display(out)
                    elif plotting_backend in [
                        'holoviews',    # bokeh
                    ]:
                        out=self._extraday_query(df).plot(backend=plotting_backend,legend=False)
                        display(out)
                    else:
                        if self._logging_level > 1: print('nothing')
                elif method == 'cufflinks':
                    if self._logging_level > 1: print('cufflinks')
                    '''
                    # cufflinks
                    https://github.com/santosjorge/cufflinks
                    
                    # params
                    help(pd.DataFrame.iplot)
                    
                    # cufflink themes
                    cufflinks.getThemes()
                    ['ggplot','pearl','solar','space','white','polar','henanigans']

                    # title placement
                    https://plotly.com/python/figure-labels/
                    https://plotly.com/python/reference/layout/#layout-title
                    '''

                    self._extraday_query(df).iplot(
                        title = {
                            'text'        : self._query,
                            'y'           : 0.025, # bottom align
                            'x'           : 0,     # left align
                            'xanchor'     : 'left',
                            'yanchor'     : 'auto',
                        },
                        legend = False,
                        #theme='ggplot'
                        margin = {'l':0,'r':0,'b':0,'t':0},
                    )
                elif method == 'bokeh':
                    if self._logging_level > 1: print('bokeh')
                    out = figure()
                    #out.plot(self._extraday_query(df).plot(backend=plotting_backend))
                    #show(out,notebook_handle=True)                
                elif method == 'px':
                    if self._logging_level > 1: print('px')
                    out = px.line(
                        self._extraday_query(df),
                        title = self._query
                    )
                    out.update_layout(showlegend=False)
                    out.show()
            else:
                if self._logging_level > 1: print('else')
                display(Text(self.name+'-'+str(self._query)+'-'+str(self._times_drawn)))

In [ ]:
# example without data
'''

a=SINGLE_PLOT(query='something')
display(a.widget)
a.update_plot()
a.update_plot()
a.update_plot()
a.update_plot()

'''
None

In [ ]:
# example with data
'''

%run DATAFRAME.ipynb

a=SINGLE_PLOT(name='name',description='des',method='cufflinks',query='open') # instantiated without data!

display(a._method)
display(a._query)
display(a._times_drawn)
display(a.widget)

a.update_plot(dat2,'px') # custom

a.update_plot(dat2) # default

'''
None

In [ ]:
'''
doesnt contain data
only explains how to plots within it generate plots off a pd.DataFrame
'''
class SINGLE_TAB:
    def __init__(self,
        name        : str              = 'SINGLE_TAB',
        description : str              = 'SINGLE_TAB_DESCRIPTION',
        method      : str              = None,
        queries     : QUERIES          = None, # either give `queries` or `plots`
        plots       : LOCATIONED_PLOTS = None,
        cols        : int              = 3,
        rows        : int              = 3,
        ):
        
        self._logging_level                    = LOGGING_LEVEL                  # GLOBAL VARIABLE

        self._times_drawn                      = 0
        self._cols                             = cols
        self._rows                             = rows
        self.name                              = name
        self.description                       = description
        self._method                           = method
        self._queries                          = self._populate_queries(queries)
        self._plots                            = self._populate_plots(plots)
        self.widget                            = self._make_widget_GridBox()

    ####################################
    # populate defaults
    ####################################
    def _populate_queries(self,queries) -> List[str] :
        if queries != None:
            return queries
        else:
            return ['query']*self._rows*self._cols

    def _populate_plots(self,plots) -> LOCATIONED_PLOTS :
        if plots != None:
            return plots 
        else: 
            # populate queries, assumes self._queries is shorter than self._rows*self._cols
            queries = [None]*self._rows*self._cols
            if self._queries != None:
                queries[:len(self._queries)] = self._queries # copy head over

            # generate single plots from queries
            idx=0
            out={}
            for r in range(self._rows):
                for c in range(self._cols):
                    out[(r,c)] = SINGLE_PLOT(
                        name   = self.name+'['+str(r)+str(c)+']',
                        method = self._method,
                        query  = queries[idx],
                    )
                    idx += 1
            return out

    def _make_widget_GridspecLayout(self) -> GridspecLayout :
        # instantiate
        gridspec = GridspecLayout(
            n_rows       = self._rows,
            n_columns    = self._cols,
            width_ratios = np.repeat(np.round(1/self._rows,0),self._rows),
        )
        
        # populate gridspec
        for r in range(self._rows):
            for c in range(self._cols):
                gridspec[r,c] = self._plots[(r,c)].widget
        
        # return
        return gridspec
    
    def _make_widget_GridBox(self) -> GridBox :
        # instantiate
        gridbox = GridBox(
            children = [x.widget for x in self._plots.values()],
            layout   = Layout(
                width                 = '100%',
                grid_template_columns = 'minmax(0, 1fr) minmax(0, 1fr) minmax(0, 1fr)',
                grid_template_rows    = 'auto auto auto',
                grid_gap              = '0px 0px',
            )
        )

        # return
        return gridbox

    # clear and draw with given data
    def update_tab(self,
        df     : pd.DataFrame = pd.DataFrame(),
        method : str          = None,
        ) -> None :
        
        if self._logging_level > 0: print(self.name,'update_tab',self._times_drawn)
            
        # ensure method populated
        if method is None: 
            method = self._method

        self._times_drawn+=1
        for plot in self._plots.values():
            plot.update_plot(df,method)


In [ ]:
# example without data
'''

a=SINGLE_TAB()
display(a.widget)
a.update_tab()
a.update_tab()
a.update_tab()
a.update_tab()

'''
None

In [ ]:
# example with data
'''

a=SINGLE_TAB(queries=['open','high','low','close','adjusted_close'])
display(a.widget)

a.update_tab(dat2,method='px')

a.update_tab(dat2,method='cufflinks')

'''
None

In [ ]:
'''
doesnt contain data
only explains how to tabs within it generate plots off a pd.DataFrame
'''
class TAB_COLLECTION:
    def __init__(self,
        name                : str              = 'TAB_COLLECTION',
        description         : str              = 'TAB_COLLECTION_DESCRIPTION',
        tabs                : TABS             = None,
        ):
        
        self._logging_level                    = LOGGING_LEVEL                  # GLOBAL VARIABLE
        
        self.name           : str              = name
        self.description    : str              = description
        self._tabs          : List[SINGLE_TAB] = tabs if tabs != None else self._populate_defaults_tabs() # contents of tab collection
        self._is_up_to_date : List[bool]       = [True]*len(self._tabs) # nothing selected by default, all tabs up to date by default
            
        # populated by self.react()
        self._method        : str              = None
        self._df            : pd.DataFrame     = pd.DataFrame()
        
        self.widget                            = self._make_widget()
        self._observers                        = [] # for observer pattern

    ####################################
    # populate defaults
    ####################################
    def _populate_defaults_tabs(self):
        return [
            SINGLE_TAB('tab1'),
            SINGLE_TAB('tab2'),
            SINGLE_TAB('tab3'),
            SINGLE_TAB('tab4'),
        ]
    
    ####################################
    # buttons
    ####################################
    '''change active tab, update as needed'''
    def _onchange_tab(self,change) -> None :
        i=self._active_tab_index
        
        # handle delayed update
        if not self._is_up_to_date[i]:
            self._log_tab_status(i,'UPDATE[DELAYED]') # logging
            self._update_tab_by_index(i)
        else:
            self._log_tab_status(i,'NO UPDATE')       # logging
        
        # logging
        self._log_update_summary()
        
    ####################################
    # build GUI
    ####################################
    def _make_widget(self) -> None :
        # make tab collection
        out=Tab(
            children=[x.widget for x in self._tabs],
            layout=Layout(width='100%',height='auto',min_height='500px')
        )

        # name tabs
        for i,tab_name in enumerate(x.name for x in self._tabs):
            out.set_title(index=i,title=tab_name)
            
        # observe active tab
        out.observe(self._onchange_tab,names='selected_index')

        # return
        return out

    ####################################
    # properties
    ####################################
    @property
    def _active_tab_index(self) -> int :
        return self.widget.selected_index
    
    @property
    def _active_tab(self) -> str :
        return self._tabs[self._active_tab_index]
    
    @property
    def _times_drawn(self) -> List[int]:
        return [x._times_drawn for x in self._tabs]
    
    ####################################
    # functions
    ####################################
    def _update_tab_by_index(self,tab_index) -> None:
        self._tabs[tab_index].update_tab(self._df,self._method) # update this tab
        self._is_up_to_date[tab_index] = True # flag as up to date
        
    '''delayed update of tab collection, update active tab immediate, delay update of non-active tabs till when they are viewed'''
    def update_collection(self,
        df     : pd.DataFrame = pd.DataFrame(),
        method : str          = None,
        ) -> None :
        self._df              = df     # remember requested plotting data
        self._method          = method # remember requested plotting method

        if len(df)==0:
            # if no data, update all tabs sequentially
            for i,tab in enumerate(self._tabs):
                self._log_tab_status(i,'UPDATE[INSTANT]') # logging
                self._update_tab_by_index(i) # update all tabs
        else:
            # else have data, only update active tab (others get updated on click)
            for i,tab in enumerate(self._tabs):
                if self._active_tab_index == i:
                    self._log_tab_status(i,'UPDATE[INSTANT]') # logging
                    self._update_tab_by_index(i) # update active tab
                else:
                    self._is_up_to_date[i] = False # flag others as pending update

        # logging
        self._log_update_summary()
    
    def _log_update_summary(self) -> None:
        if self._logging_level > 0: 
            print(self.name,':','compute times',self._times_drawn)
            print(self.name,':','is up to date',self._is_up_to_date)
        
    def _log_tab_status(self,tab_index,comment : str):
        if self._logging_level > 0: 
            print(self.name,':',comment,'INDEX',tab_index,'NAME',self._tabs[tab_index].name)

In [ ]:
# example with no data
'''

a=TAB_COLLECTION()

display(a.widget)

a.update_collection()

a.update_collection(dat2) # update on request

a.update_collection(dat2,method='cufflinks') # update on request

'''
None

In [ ]:
# example with data
'''

a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB(name='a',queries=['open']),
        SINGLE_TAB(name='b',queries=['close']),
        SINGLE_TAB(name='c',queries=['high']),
        SINGLE_TAB(name='d',queries=['low']),
    ]
)
display(a.widget)

a.update_collection(dat2,method='px')

a.update_collection(dat2,method='cufflinks')

'''
None

In [ ]:
# test delayed updating
'''

a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB('a'),
        SINGLE_TAB('b'),
        SINGLE_TAB('c'),
        SINGLE_TAB('d'),
    ]
)
a.widget

# global delayed update
a.update_collection()

# request immediate update of specific tab
a._tabs[0].update_tab('a')

a._tabs[0].update_tab('a')

a._tabs[2].update_tab('a')

a._tabs[2].update_tab('a')

a._tabs[3].update_tab('a')

a._times_drawn

'''
None